In [4]:
!pip install pydantic

In [1]:
# 새로운 json 파일 생성 추가

import json
from pathlib import Path
import shutil
from shutil import copytree, Error
from tqdm import tqdm
import os


json_data = []


try:
    with open('printed_data_info.json', 'rt', encoding='UTF8') as json_file:
        try:
            json_data = json.load(json_file)

        except ValueError as ev:
            print("ValueError: {0}".format(ev))
        
        train_font_list = ['휴먼명조', '휴먼고딕', '전북', '제주 명조', 'EBS훈민정음새론', '본고딕', 'KBIZ 한마음고딕', '함초롱돋움', '나눔바른고딕', '티몬소리', '맑은고딕', '태룡고딕', '한겨레', '함초롱바탕', '가는안상수체', '노토산스', '서울한강', '바탕', '서울남산', '미생', '나눔고딕 코딩', '조선명조', '돋움', '수화명조', '나눔명조']
        
        # img 분류하기
        text_list = []

        for i in tqdm(range(0, len(json_data["annotations"])), desc="파일 분류 진행률"):
            # 현재 확보한 파일 = 00000000.png ~ 00532658.png
            if(int(json_data["annotations"][i]["image_id"]) > 0 and int(json_data["annotations"][i]["image_id"]) <= 532658 ):
                # 추출한 25개의 폰트에 해당하면
                if json_data["annotations"][i]["attributes"]["font"] in train_font_list:
                    # 해당하는 text를 list에 저장
                    text = json_data["annotations"][i]["text"]
                    text_list.append(text)

        text_set = set(text_list) #집합set으로 변환
        new_text_list = list(text_set) #list로 변환


        number = [] # 폰트 개수 리스트

        # number리스트에 text_list에 저장된 text별 개수를 저장
        for text in tqdm(new_text_list, desc="폰트 개수 list 생성"):
            number.append(text_list.count(text))

        # text와 개수 매칭 시키기
        dict_text = {} # text와 폰트 개수 매칭 시킬 딕셔너리           
        dict_text = dict(zip(new_text_list, number)) # text와 폰트 개수를 dictionary로 만들기

        # 25개 폰트 모두 가진 text 담기
        text25 = []

        for text, num in tqdm(dict_text.items(), desc="25개 폰트 모두 가진 text 담기"):
            if num == 25:
                text25.append(text)

        print(len(text25))
        
        # id값과 file_name 매칭 시키기
        dict_file = {} # img id와 img 파일 네임을 매칭 시킬 딕셔너리
        img_id = [] # img id list
        file_name = [] # img file name list
        
        for i in range(0, len(json_data["images"])):
            img_id.append(json_data["images"][i]["id"])
            file_name.append(json_data["images"][i]["file_name"])
            
        dict_file = dict(zip(img_id, file_name)) # id와 file_name을 dictionary로 만들기
        
        # 새로운 json 파일 생성
        train_data = []
        train_data = json.loads(json.dumps(train_data))
        
        for text in text25:
            train_data.append({"text": text, "origin":"", "target": []})
        
        # img 분류하기        
        for i in tqdm(range(0, len(json_data["annotations"])), desc="파일 복사 진행률"):
            # 폴더 생성
            if json_data["annotations"][i]["attributes"]["font"] in train_font_list:
                path = Path('syllable/train_data/' + json_data["annotations"][i]["attributes"]["font"])
                path.mkdir(exist_ok=True) # FileExistsError 무시

                try:
                    # 한 글자(음절) 데이터만 분류
                    if json_data["annotations"][i]["text"] in text25:
                        src = 'syllable/' + dict_file.get(json_data["annotations"][i]["image_id"]) # 기존에 파일이 있는 위치
                        dst = path # 복사할 위치 (폰트 별 폴더)
                        if os.path.exists(src):
                            shutil.copy(src, dst)
                            for j in range(0, len(train_data)): # 생성한 json파일 만큼 반복
                                if train_data[j]['text'] is json_data["annotations"][i]["text"]: # 파일 복사 진행중인 글자가 json 파일의 text와 같으면
                                    if json_data["annotations"][i]["attributes"]["font"] == "맑은고딕": # 해당 글자가 맑은 고딕이면
                                        print(str(dst))
                                        train_data[j]["origin"] = str(dst)
                                    else: # 해당 글자가 맑은 고딕이 아니면
                                        train_data[j]["target"].append(str(dst))
                            
                except Error as err:
                    errors.extend(err.args[0])
        
        # 생성된 json 파일 출력
        print(json.dumps(train_data, ensure_ascii=False, indent="\t"))
        
        with open('train_data.json', 'w') as outfile:
            json.dump(train_data, outfile, ensure_ascii=False)
        
except OSError as err:
    print("OS error: {0}".format(err))

25개 폰트 모두 가진 text 담기: 100%|█████████████████████████████████████████████████| 11172/11172 [00:00<00:00, 2772543.89it/s]


3380


파일 복사 진행률:   0%|                                                                            | 0/878100 [00:00<?, ?it/s]

syllable\train_data\맑은고딕


파일 복사 진행률:   0%|                                                                | 318/878100 [00:00<04:38, 3149.43it/s]

...

파일 복사 진행률:   9%|█████▌                                                        | 79658/878100 [00:25<04:08, 3217.27it/s]

syllable\train_data\맑은고딕


파일 복사 진행률: 100%|█████████████████████████████████████████████████████████████| 878100/878100 [03:20<00:00, 4381.70it/s]


[
	{
		"text": "겊",
		"origin": "",
		"target": [
			"syllable\\train_data\\나눔고딕 코딩"
		]
	},
	{
		"text": "뤮",
		"origin": "",
		"target": [
			"syllable\\train_data\\수화명조"
		]
	},
	{
		"text": "낃",
		"origin": "",
		"target": [
			"syllable\\train_data\\KBIZ 한마음고딕"
		]
	},
	{
		"text": "슐",
		"origin": "",
		"target": [
			"syllable\\train_data\\조선명조"
		]
	},
	{
		"text": "퀘",
		"origin": "",
		"target": [
			"syllable\\train_data\\제주 명조"
		]
	},
	{
		"text": "쌠",
		"origin": "",
		"target": [
			"syllable\\train_data\\서울남산"
		]
	},
	{
		"text": "몽",
		"origin": "",
		"target": [
			"syllable\\train_data\\티몬소리"
		]
	},
	{
		"text": "릸",
		"origin": "",
		"target": [
			"syllable\\train_data\\태룡고딕"
		]
	},
	{
		"text": "퇅",
		"origin": "",
		"target": [
			"syllable\\train_data\\조선명조"
		]
	},
	{
		"text": "켽",
		"origin": "",
		"target": [
			"syllable\\train_data\\한겨레"
		]
	},
	{
		"text": "놓",
		"origin": "",
		"target": [
			"syllable\\train_data\\수화명조"
		]
	},
	{
		"text": "제",
		"

]
